In [ ]:
#hide
#Run once per session
!pip install fastai wwf -q --upgrade
!pip install rasterio
!pip install geopandas

     |████████████████████████████████| 549 kB 17.4 MB/s 
     |████████████████████████████████| 182 kB 71.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 20.9 MB 1.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 33.3 MB/s 
     |████████████████████████████████| 16.6 MB 68.4 MB/s 
     |████████████████████████████████| 7.8 MB 57.5 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import os
from PIL import Image
import numpy as np
from typing import Tuple
import rasterio as rio
import geopandas as gpd

from fastai.basics import *
from fastai.vision.all import *
from fastai.vision.core import *
from fastai.vision.data import *
from fastai.data.all import *
from fastcore.xtras import Path
from fastai.callback.hook import summary
from fastai.callback.progress import ProgressCallback
from fastai.callback.schedule import lr_find, fit_flat_cos
from fastai.data.block import DataBlock
from fastai.data.external import untar_data, URLs
from fastai.data.transforms import get_image_files, FuncSplitter, Normalize
from fastai.layers import Mish
from fastai.losses import BaseLoss
from fastai.optimizer import ranger
from fastai.torch_core import tensor
from fastai.vision.augment import aug_transforms
from fastai.vision.core import PILImage, PILMask
from fastai.vision.data import ImageBlock, MaskBlock, imagenet_stats
from fastai.vision.learner import unet_learner


from torch import nn
from torchvision.models.resnet import resnet34
from torchvision.models.vgg import vgg16, vgg16_bn
import torch
import torch.nn.functional as F




In [ ]:
#Useful functions

def open_npy(fn, chans=None):
    im = torch.from_numpy(np.load(str(fn)))
    if chans is not None: im = im[chans]
    return im

def open_geotiff(fn, chans=None):
    with rio.open(str(fn)) as f:
        data = f.read()
        data = data.astype(np.float32)
    im = torch.from_numpy(data)
    if chans is not None: im = im[chans]
    return im

class MultiChannelTensorImage(TensorImage):
    _show_args = ArrayImageBase._show_args
    def show(self, channels=[1], ctx=None, vmin=None, vmax=None, **kwargs):
        "These need refactoring maybe"
        if channels == 'spectra':
            return show_mean_spectra(self, ctx=ctx,  **kwargs)
        if len(channels) == 3:
            return show_composite(self, channels=channels, ctx=ctx, vmin=vmin, vmax=vmax,
                                  **{**self._show_args, **kwargs})
        if len(channels) == 2:
            return show_normalized_spectral_index(self, channels=channels, ctx=ctx,
                                                  **{**self._show_args, **kwargs})
        elif len(channels) == 1:
            return show_single_channel(self, channel=channels[0], ctx=ctx,
                                       **{**self._show_args, **kwargs})

    @classmethod
    def create(cls, fn:(Path,str,ndarray), chans=None,  **kwargs) ->None:
        if isinstance(fn, Tensor): fn = fn.numpy()
        if isinstance(fn, ndarray):
            im = torch.from_numpy(fn)
            if chans is not None: im = im[chans]
            return cls(im)
        if isinstance(fn, Path) or isinstance(fn, str):
            if str(fn).endswith('npy'): return cls(open_npy(fn=fn, chans=chans))
            elif str(fn).endswith('.tif'): return cls(open_geotiff(fn=fn, chans=chans))

    def __repr__(self): return f'{self.__class__.__name__} size={"x".join([str(d) for d in self.shape])}'

MultiChannelTensorImage.create = Transform(MultiChannelTensorImage.create)

def show_composite(img, channels, ax=None, figsize=(3,3), title=None, scale=True,
                   ctx=None, vmin=None, vmax=None, scale_axis=(0,1), **kwargs)->plt.Axes:
    "Show three channel composite so that channels correspond to R, G and B"
    ax = ifnone(ax, ctx)
    if ax is None: _, ax = plt.subplots(figsize=figsize)
    r, g, b = channels
    tempim = img.data.cpu().numpy()
    im = np.zeros((tempim.shape[1], tempim.shape[2], 3))
    im[...,0] = tempim[r]
    im[...,1] = tempim[g]
    im[...,2] = tempim[b]

    if scale: im = norm(im, vmin, vmax, scale_axis)
    ax.imshow(im, **kwargs)
    ax.axis('off')
    if title is not None: ax.set_title(title)
    return ax

def show_single_channel(img, channel, ax=None, figsize=(3,3), ctx=None,
                        title=None, **kwargs) -> plt.Axes:
    ax = ifnone(ax, ctx)
    if ax is None: _, ax = plt.subplots(figsize=figsize)
    tempim = img.data.cpu().numpy()
    ax.imshow(norm(tempim[channel], vmin=tempim[channel].min(), vmax=tempim[channel].max()), **kwargs)
    ax.axis('off')
    if title is not None: ax.set_title(title)
    return ax

def show_normalized_spectral_index(img, channels, ax=None, figsize=(3,3), ctx=None,
                                   title=None, **kwargs) -> plt.Axes:
    "Show normalized spectral index such as NDVI"
    ax = ifnone(ax, ctx)
    if ax is None: _, ax = plt.subplots(figsize=figsize)
    b_0, b_1 = channels
    tempim = img.data.cpu().numpy()
    im = (tempim[b_0] - tempim[b_1])/(tempim[b_0] + tempim[b_1])
    ax.imshow(im, vmin=-1, vmax=1, **kwargs)
    ax.axis('off')
    if title is not None: ax.set_title(title)
    return ax

def show_mean_spectra(img, ax=None, figsize=(3,3), ctx=None, title=None, **kwargs) -> plt.Axes:
    "Show average spectra graph"
    ax = ifnone(ax, ctx)
    if ax is None: _, ax = plt.subplots(figsize=figsize)
    tempim = img.data.cpu().numpy()
    means = np.nanmean(tempim, axis=(-2, -1))
    ax.plot(means, **kwargs)
    ax.grid(True)
    if title is not None: ax.set_title(title)
    return ax

def norm(vals, vmin=None, vmax=None, axis=(0,1)):
    """
    For visualization purposes scale image with `(vals-vmin)/(vmax-vmin),
    with vmin and vmax either specified or within 0.01 and 0.99 quantiles of all values
    """
    vmin = ifnone(vmin, np.quantile(vals, 0.01, axis=axis))
    vmax = ifnone(vmax, np.quantile(vals, 0.99, axis=axis))
    ret_im = (vals - vmin)/(vmax-vmin)
    ret_im[ret_im < 0] = 0
    ret_im[ret_im > 1] = 1
    return ret_im

In [ ]:
#export
def MultiChannelImageBlock(cls=MultiChannelTensorImage, chans=None):
    "Default behaviour: use all channels"
    return TransformBlock(partial(cls.create, chans=chans))

In [ ]:
class TifSegmentationDataLoaders(DataLoaders):
    "Needs a better name"
    @classmethod
    @delegates(DataLoaders.from_dblock)
    def from_label_funcs(cls, path, fnames, label_func, chans=None,
                         extensions=['.tif'], valid_pct=0.2, seed=None,
                         codes=None, item_tfms=None, batch_tfms=None, **kwargs):
        "Create from list of `fnames` in `path`s with `label_func`."
        dblock = DataBlock(blocks=(MultiChannelImageBlock(chans=chans),
                                   MaskBlock(codes=codes)),
                           splitter=RandomSplitter(valid_pct, seed=seed),
                           get_y=label_func,
                           item_tfms=item_tfms,
                           batch_tfms=batch_tfms)
        res = cls.from_dblock(dblock, fnames, path=path, **kwargs)
        return res

In [ ]:
path = Path("/content/drive/MyDrive/CNN/Data/2020")

In [ ]:
path_im = os.path.join(path,'chips')
path_lbl = os.path.join(path,'lable')

In [ ]:
fnames = get_image_files(path/'chips')
lbl_names = get_image_files(path/'lable')

In [ ]:
get_msk = lambda img_fn: os.path.join(path_lbl , "_".join(['lable'] + (img_fn.stem).split("_")[1:]) + img_fn.suffix)

In [ ]:
codes = np.array(["no_val","water", "urban", "vegitation", "barren"])
name2id = {v:k for k,v in enumerate(codes)}
print(name2id)



In [ ]:
void_code = name2id['no_val']
def acc_camvid(inp, targ):
  targ = targ.squeeze(1)
  mask = targ != void_code
  return (inp.argmax(dim=1)[mask]==targ[mask]).float().mean()

In [ ]:
segm = TifSegmentationDataLoaders.from_label_funcs(path=path, bs=4, codes=codes,
                                                   fnames=fnames,
                                                   label_func = get_msk)

In [ ]:
#Set n_in = 9 for landsat 8 data and n_in = 7 for landsat 5 data

opt = adam #optimization function 
learn2 = unet_learner(segm, vgg16_bn, n_in=9, pretrained=False,  metrics=acc_camvid, self_attention=True, act_cls=Mish, opt_func=opt)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
learn2.fit_one_cycle(20, lr_max=1e-3)

epoch,train_loss,valid_loss,acc_camvid,time
0,1.469178,0.893530,0.646422,01:01
1,0.936734,0.473043,0.814069,00:51
2,0.723660,0.407208,0.823296,00:51
3,0.605489,0.418429,0.843198,00:51
4,0.529001,0.525409,0.789738,00:51
5,0.482360,0.290842,0.882096,00:51
6,0.437495,0.462222,0.791995,00:51
7,0.367434,0.277217,0.885173,00:51
8,0.333185,0.259931,0.889339,00:51
9,0.305951,0.257231,0.889336,00:51


In [ ]:
tif_paths = []
for root, dirs, files in os.walk("/content/drive/MyDrive/VB/CNN/prediction_data/2020"):
    for name in files:
        if name.endswith((".tif")):
            full_path = os.path.join(root, name)
            tif_paths.append(full_path)
print(len(tif_paths))

384


In [ ]:
#os.makedirs("/content/drive/MyDrive/VB/CNN/prediction_results_cnn2/2020")

for path_to_img in tif_paths:

  pred2 = learn2.predict(MultiChannelTensorImage.create(path_to_img))
  pred_rio_img = rio.open(path_to_img)
  meta = pred_rio_img.meta
  meta.update({'count': 1})

  classification_cnn = np.zeros((1,256, 256))
  classification_cnn[0] = pred2[0]

  a = path_to_img.split("/")[-1].split("_")
  a[0] = "pred"
  out_file_name = "_".join(a)

  with rio.open(f'/content/drive/MyDrive/VB/CNN/prediction_results_cnn2/2020/{out_file_name}', 'w', **meta) as dst:
      dst.write(classification_cnn)